In [17]:
import requests
import boto3
from mypy_boto3_s3 import S3Client
import json

In [20]:
API_BASE_URL="http://localhost:8000"
BUCKET="cow-detect-maia"

app_info = requests.get(f"{API_BASE_URL}/app-info").json()
model_name = app_info["model_info"]["model_metadata"]["name"]
# region = {"HerdNet": "Chad-HN", "FasterRCNNResNetFPN": "Chad-FRC"}[model_name]
region = "Time-Measurement"

print(f"{model_name=}  region: {region}")


model_name='FasterRCNNResNetFPN'  region: Time-Measurement


In [19]:
from pprint import pprint

pprint(app_info)

{'model_info': {'cfg_path': 'configs/test/faster_rcnn.yaml',
                'model_metadata': {'backbone_arch': 'resnet50',
                                   'cfg_path': 'configs/test/faster_rcnn.yaml',
                                   'class_name': 'LossWrapper',
                                   'name': 'FasterRCNNResNetFPN',
                                   'num_classes': 7,
                                   'patch_size': 512,
                                   'trainable_backbone_layers': 4,
                                   'weights_path': 'data/models/faster-rcnn/resnet50-100-epochs-tbl4/best_model.pth'},
                'weights_path': 'data/models/faster-rcnn/resnet50-100-epochs-tbl4/best_model.pth'},
 's3_bucket': 'cow-detect-maia'}


In [10]:
session = boto3.Session(region_name="us-east-1", profile_name="dvc-user")
s3: S3Client = session.client("s3")



In [24]:
prefix = f"{region}/2025-11-22/"
s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix)['Contents'][0]['Key']

'Time-Measurement/2025-11-22/2af4192ee4de23185abb0e1677b61fd9ea52db61.JPG'

In [26]:
payload = {"s3_path": f"s3://{BUCKET}/Time-Measurement/2025-11-22/2af4192ee4de23185abb0e1677b61fd9ea52db61.JPG"}
resp = requests.post(f"{API_BASE_URL}/predict", json=payload)